# First version of AE with clustering loss

## Import useful modules

In [1]:
!pip install scanpy
!pip install keras
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip3 install igraph
!pip3 install leidenalg
!pip install os
!pip install csv
!pip install random

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement csv (from versions: none

In [ ]:
from keras import layers
from keras.layers import Layer
from keras.models import load_model, Model
from keras import backend as K
from keras.losses import KLDivergence
import numpy as np
import scanpy as sc
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
from layers import ConstantDispersionLayer, SliceLayer, ColWiseMultLayer
import keras
from keras.layers import Layer
from sklearn.model_selection import train_test_split
import datetime
import time
import os
import csv
from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn.metrics.pairwise import euclidean_distances
import math
import random

2024-06-18 20:52:18.333692: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 20:52:19.276829: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 20:52:19.563556: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 20:52:27.729029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
tf.__version__

In [ ]:
keras.__version__

## Preprocessing

### Load dataset

On importe le dataset baron et on applique le prétraitement habituel

In [ ]:
filename="./dataset/barondata_rawcounts.csv"
data_baron=pd.read_csv(filename, sep=';', comment="#", index_col=0)
data_baron

In [ ]:
filename="./dataset/barondata_annotations.csv"
annotations_baron=pd.read_csv(filename, sep=';', comment="#", index_col=0)
annotations_baron

In [ ]:
data_ann=sc.AnnData(data_baron.T)
data_ann

In [ ]:
data_ann.obs['label'] = annotations_baron['label']

### Filter data

On filtre les données de manière peu stricte

In [ ]:
sc.pp.filter_cells(data_ann, min_genes=1)
sc.pp.filter_genes(data_ann, min_cells=1)

In [ ]:
data_ann.raw = data_ann.copy()

In [ ]:
data_ann

### Normalize and scale

In [ ]:
sc.pp.normalize_total(data_ann)
data_ann.obs['size_factors'] = data_ann.obs.n_genes / np.median(data_ann.obs.n_genes)
#data_ann.obs['size_factors'] = 1.0
sc.pp.log1p(data_ann)


In [ ]:
sc.pp.highly_variable_genes(data_ann, n_top_genes=2000, batch_key="label")
sc.pl.highly_variable_genes(data_ann)

In [ ]:
sc.pp.scale(data_ann)

### Explore the result

In [ ]:
data_ann.X

In [ ]:
data_ann.n_vars

In [ ]:
data_ann.X

In [ ]:
data_ann.var

In [ ]:
data_ann.obs

In [ ]:
data_ann.raw.X

In [ ]:
highly_variable_genes = data_ann.var[data_ann.var['highly_variable']].index.tolist()
count_data_hvg = data_ann[:, highly_variable_genes].X
count_data_hvg=count_data_hvg.toarray()
count_data_hvg

In [ ]:
raw_count_hvg=data_ann.raw[:,highly_variable_genes].X

## Create autoencoder

In [ ]:
def create_contrastive_encoder(input_dim, dims, dropout=0.8):
    Inputs = layers.Input(shape=(input_dim,))
    x = layers.Dropout(rate=dropout)(Inputs)  # L'argument input_shape n'est pas nécessaire ici
    for i in range(len(dims) - 1):
        x = layers.Dense(dims[i], kernel_initializer='glorot_uniform')(x)
        x = layers.BatchNormalization(epsilon=1e-5, momentum=0.1)(x)
        x = layers.ReLU()(x)
    
    # Dernière couche sans activation pour obtenir les embeddings
    x = layers.Dense(dims[-1],  kernel_initializer='glorot_uniform')(x)
    x = layers.Lambda(lambda y: tf.nn.l2_normalize(y, axis=1))(x)
    model = Model(Inputs, x)
    return model

In [ ]:
def SupConLoss(features, labels=None, mask=None, temperature=0.07, base_temperature=0.07, contrast_mode='all'): 
    if len(features.shape) < 3:
        raise ValueError('`features` needs to be [bsz, n_views, ...], at least 3 dimensions are required')
    if len(features.shape) > 3:
        features = tf.reshape(features, [tf.shape(features)[0], tf.shape(features)[1], -1]) 
    batch_size = tf.shape(features)[0]
    if labels is not None and mask is not None:
        raise ValueError('Cannot define both `labels` and `mask`')
    elif labels is None and mask is None:
        mask = tf.eye(batch_size, dtype=tf.float32)
    elif labels is not None:
        labels = tf.reshape(labels, (-1, 1))
        if labels.shape[0] != batch_size:
            raise ValueError('Num of labels does not match num of features')
        mask = tf.cast(tf.equal(labels, tf.transpose(labels)), dtype=tf.float32)
    else:
        mask = tf.eye(batch_size, dtype=tf.float32)
    contrast_count = features.shape[1]
    contrast_feature = tf.reshape(tf.concat(tf.unstack(features, axis=1), axis=0), [-1, tf.shape(features)[2]])
    if contrast_mode == 'one':
        anchor_feature = features[:, 0]
        anchor_count = 1
    elif contrast_mode == "all":
        anchor_feature = contrast_feature
        anchor_count = contrast_count
    else:
        raise ValueError(f'Unknown mode : {contrast_mode}')
        
    anchor_dot_contrast = tf.math.divide(tf.linalg.matmul(anchor_feature, contrast_feature, transpose_b=True), temperature)
    logits_max = tf.reduce_max(anchor_dot_contrast, axis=1, keepdims=True)
    logits = anchor_dot_contrast - logits_max
    mask = tf.tile(mask, [anchor_count, contrast_count])
    logits_mask = tf.ones_like(mask) - tf.eye(batch_size * anchor_count, dtype=tf.float32)
    mask *= logits_mask
    exp_logits = tf.exp(logits) * logits_mask
    log_prob = logits - tf.math.log(tf.reduce_sum(exp_logits, axis=1, keepdims=True))

    mean_log_prob_pos = tf.reduce_sum(mask * log_prob, axis=1) / tf.reduce_sum(mask, axis=1)

    loss = -(temperature / base_temperature) * mean_log_prob_pos
    loss = tf.reshape(loss, [anchor_count, batch_size])
    loss = tf.reduce_mean(loss)

    return loss

In [ ]:
def auto_kmeans(encoder, x_counts, obs,n='auto',  plot=False):
    y=obs
    ari=[]
    nmi=[]
    ca=[]
    x=[]
    if n=="auto":
        for n in range (1,20):
            kmeans=KMeans(n_clusters=n, n_init=30, verbose=0)
            y_pred=kmeans.fit_predict(encoder.predict(x_counts, verbose=0))
            ari.append(adjusted_rand_score(y, y_pred))
            nmi.append(normalized_mutual_info_score(y, y_pred))
            ca.append(cluster_acc(y, y_pred))
            x.append(n)
        somme_metriques = [x + y + z for x, y, z in zip(ari, nmi, ca)]
        n=(somme_metriques.index(max(somme_metriques))+1)
    
    kmeans=KMeans(n_clusters=n, n_init=20)
    y_pred=kmeans.fit_predict(encoder.predict(x_counts))
    
    if plot==True:
        predict_data=encoder.predict(x_counts)
        adata_latent = sc.AnnData(predict_data)
        obs_df = pd.DataFrame({'label': y})
        adata_latent.obs=obs_df
        adata_latent.obs["kmeans"]=y_pred
        sc.pp.neighbors(adata_latent)
        sc.tl.umap(adata_latent)
        sc.pl.umap(adata_latent, color='label')
        plt.plot(x,ari)
        plt.plot(x,nmi)
        plt.plot(x,ca)
        plt.xlabel("nombre de clusters")
        plt.ylabel("Score")
        plt.legend(labels=["ARI", "NMI", "CA"])
        ari = adjusted_rand_score(y, y_pred)
        print("Indice de Rand ajusté (ARI) :", ari)
        nmi = normalized_mutual_info_score(y, y_pred)
        print("Normalized mutual info (NMI) :", nmi)
        ca=cluster_acc(y, y_pred)
        print("Clustering accuracy (CA) :", ca)
        sc.pl.umap( adata_latent, color=["kmeans"], legend_loc="on data")
        plt.plot()
    return y_pred, n, kmeans.cluster_centers_

def auto_leiden(encoder, x_counts, y, res="auto", plot=False):
    predict_data=encoder.predict(x_counts, verbose=0)
    adata_latent = sc.AnnData(predict_data)
    obs_df = pd.DataFrame({'label': y})
    adata_latent.obs=obs_df
    sc.pp.neighbors(adata_latent, use_rep='X')
    sc.tl.umap(adata_latent)
    list_ari=[]
    list_nmi=[]
    list_ca=[]
    x=[]
    
    if res=="auto":
        #search for the best resolution
        for i in range (1,10):
            sc.tl.leiden(adata_latent, key_added="leiden", resolution=i/100)
            predict_cluster=adata_latent.obs["leiden"]
            list_ari.append(adjusted_rand_score(y, predict_cluster))
            list_nmi.append(normalized_mutual_info_score(y, predict_cluster))
            list_ca.append(cluster_acc(y, predict_cluster))
            x.append(i/100)
        for i in range (1,11):
            sc.tl.leiden(adata_latent, key_added="leiden", resolution=i/10)
            predict_cluster=adata_latent.obs["leiden"]
            list_ari.append(adjusted_rand_score(y, predict_cluster))
            list_nmi.append(normalized_mutual_info_score(y, predict_cluster))
            list_ca.append(cluster_acc(y, predict_cluster))
            x.append(i/10)
        somme_metriques = [x + y + z for x, y, z in zip(list_ari, list_nmi, list_ca)]
        res=x[somme_metriques.index(max(somme_metriques))]
        print("La résolution est de : ", res)
    #compute for the best resolution
    sc.tl.leiden(adata_latent, key_added="leiden_res_%.4f" % (res), resolution=res)
    predict=adata_latent.obs["leiden_res_%.4f" % (res)]
    
    
    #compute cluster center for initialization
    init_pred=np.asarray(predict,dtype=int)
    features=pd.DataFrame(adata_latent.X,index=np.arange(0,adata_latent.shape[0]))
    Group=pd.Series(init_pred,index=np.arange(0,adata_latent.shape[0]),name="Group")
    Mergefeature=pd.concat([features,Group],axis=1)
    cluster_centers=np.asarray(Mergefeature.groupby("Group").mean())
    n_clusters=len(np.unique(init_pred))
    
    #set of plot if required
    if plot==True:
        sc.pl.umap(adata_latent, color='label')
        plt.plot(x,list_ari)
        plt.plot(x,list_nmi)
        plt.plot(x,list_ca)
        plt.xlabel("nombre de clusters")
        plt.ylabel("Score")
        plt.legend(labels=["ARI", "NMI", "CA"])
        plt.plot()
        sc.pl.umap( adata_latent, color=["leiden_res_%.4f" % (res)], legend_loc="on data")
        ari = adjusted_rand_score(y, predict)
        print("Indice de Rand ajusté (ARI) :", ari)
        nmi = normalized_mutual_info_score(y, predict)
        print("Normalized mutual info (NMI) :", nmi)
        ca=cluster_acc(y, predict)
        print("Clustering accuracy (CA) :", ca)
 
        #crosstab = pd.crosstab(predict,y)
        #sns.heatmap(crosstab, annot=True, cmap='Blues')
        #plt.ylabel('Clusters prédits')
        #plt.xlabel('Annotations réelles')
        #plt.title('Matrice de confusion')
        #plt.show()
    return res, predict, n_clusters, cluster_centers

def cluster_embedding(model, X, y, method=None, res='auto', n='auto', plot=False):
    if method=='leiden':
        res, predict, n_clusters, cluster_centers=auto_leiden(encoder=model, x_counts=X, y=y, res=res, plot=plot)
        return res, predict, n_clusters, cluster_centers
    elif method=='kmeans':
        y_pred, n_max, cluster_centers=auto_kmeans(encoder=model, x_counts=X, obs=y,n=n,  plot=plot)
        return res, y_pred, n_max, cluster_centers
    else:
        raise ValueError(f'Unknown method : {method}')
    

In [ ]:
def cluster_acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = np.asarray(y_true)
   
    y_pred = np.asarray(y_pred)
    
    # Assurez-vous que les étiquettes sont de type str
    y_true= y_true.astype(str)
    y_pred = y_pred.astype(str)
    
    # Trouver les étiquettes uniques
    labels = np.unique(np.concatenate((y_true, y_pred)))
    n_labels = len(labels)

    # Construire la matrice de coût (matrice de confusion)
    cost_matrix = np.zeros((n_labels, n_labels), dtype=int)
    for i, label_true in enumerate(labels):
        for j, label_pred in enumerate(labels):
            cost_matrix[i, j] = np.sum((y_true == label_true) & (y_pred == label_pred))

    # Résoudre le problème de correspondance bipartite optimal
    row_ind, col_ind = linear_assignment(cost_matrix.max() - cost_matrix)

    # Calculer la précision
    accuracy = np.sum([cost_matrix[i, j] for i, j in zip(row_ind, col_ind)]) / y_true.size

    return accuracy


In [ ]:
def save_results_to_csv(history, filename):
    file_exists = os.path.isfile(filename)
    with open(filename, mode='a') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(history.keys())
        writer.writerow(history.values())
        
def check_existing_filename(filename):
    base, ext = os.path.splitext(filename)
    counter = 1
    while os.path.exists(filename):
        filename = f"{base}_{counter}{ext}"
        counter += 1
    return filename

def save_plot_umap(model, x, y, y_pred, res, epoch, pdf_pages, train_val="train"):
    #Récupération des données et projection dans l'espace latent 
    predict_data=model.predict(x, verbose=0)
    obs_df = pd.DataFrame({'label': y})
    
    #Préparation des données pour Scanpy
    adata_latent = sc.AnnData(X=predict_data)
    adata_latent.obs = obs_df
    adata_latent.obs['predict'] = y_pred.astype(str)

    sc.pp.neighbors(adata_latent, use_rep='X')
    sc.tl.umap(adata_latent)
    
    # Génération du UMAP avec Scanpy
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    sc.pl.umap(adata_latent, color='label', ax=axs[0], show=False)
    axs[0].set_title(f'UMAP projection - Labels ({train_val}) (Res: {res}, Iter: {epoch})')
    sc.pl.umap(adata_latent, color='predict', ax=axs[1], show=False)
    axs[1].set_title(f'UMAP projection - Predictions ({train_val}) (Res: {res}, Iter: {epoch})')

    pdf_pages.savefig(fig)
    plt.close(fig)

In [ ]:
def split(x_counts, y):
    train_idx, temp_idx = train_test_split(np.arange(len(y)), stratify=y, test_size=0.3, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, stratify=y[temp_idx], test_size=0.5, random_state=42)
    x_train=x_counts[train_idx]
    x_val=x_counts[val_idx]
    x_test=x_counts[test_idx]
    y_train=y[train_idx]
    y_val=y[val_idx]
    y_test=y[test_idx]
    print("Size of train set : ", x_train.shape)
    print("Size of val set : ", x_val.shape)
    print("Size of test set : ", x_test.shape)
    return x_train, x_val, x_test, y_train, y_val, y_test

In [ ]:
x_train, x_val, x_test, y_train, y_val, y_test = split(count_data_hvg, data_ann.obs['label'])

In [ ]:
def sample_hyperparameters(lr_space, dropout_space, batch_size_space, noise_space, temperature_space):
    lr=random.choice(lr_space)
    dropout=random.choice(dropout_space)
    batch_size=random.choice(batch_size_space)
    noise=random.choice(noise_space)
    temperature=random.choice(temperature_space)
    return lr, dropout, batch_size, noise, temperature, 

In [ ]:
def cosine_scheduler(epoch, lr, epochs=500, lr_decay_rate=0.1):
    eta_min = lr * (lr_decay_rate ** 3)
    new_lr = eta_min + (lr - eta_min) * (1 + math.cos(math.pi * epoch / 500)) / 2
    return new_lr

In [ ]:
def train_model(X, Y=None, n='auto', res='auto', nb_epochs=30, lr=0.4, 
                temperature=0.07, dropout=0.9, evaluate_training=False, eval_interval=1, 
                layers=[256, 64, 32], noise=0, method='leiden', batch_size=200, 
               X_test=None, Y_test=None, early_stopping=True, plot_loss=False):
    dims = [X.shape[1]]+layers
    model= create_contrastive_encoder(input_dim=X.shape[1], dims=layers, dropout=dropout)
    model.summary()
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
    val_losses=[]
    losses=[]
    idx=np.arange(len(X))
    idx_test=np.arange(len(X_test))
    
    #Création du dictionnaire pour le monitoring
    history={
        "dropout": dropout,
        "noise": noise, 
        "lr": lr,
        'batch_size' : batch_size,
        'temperature' : temperature, 
        "loss":[],
        "val_loss":[]
    }

    
    for epoch in range (nb_epochs):
        lr = cosine_scheduler(epoch, lr, epochs=nb_epochs, lr_decay_rate=0.1)
        optimizer.learning_rate.assign(lr)
        np.random.shuffle(idx)
        np.random.shuffle(idx_test)
        loss_=0
        val_loss_=0
        for pre_index in range (len(X)//batch_size+1):
            c_idx=np.arange(pre_index*batch_size, min(len(X), (pre_index+1)*batch_size))
            if len(c_idx)==0:
                continue
            c_idx=idx[c_idx]
            c_inp=X[c_idx]
            if noise is None or noise == 0 :
                input1=tf.convert_to_tensor(c_inp, dtype=tf.float32)
                input2=tf.convert_to_tensor(c_inp, dtype=tf.float32)
            else : 
                noise_vec = np.random.normal(loc=0, scale=noise, size=c_inp.shape)
                input1 = tf.convert_to_tensor(c_inp + noise_vec, dtype=tf.float32)
                noise_vec = np.random.normal(loc=0, scale=noise, size=c_inp.shape)
                input2 = tf.convert_to_tensor(c_inp + noise_vec, dtype=tf.float32)
            
            with tf.GradientTape() as tape :
                anchors_output=model(input1,  training=True)
                neighbors_output=model(input2,  training=True)
                features = tf.stack([anchors_output, neighbors_output], axis=1)
                total_loss= SupConLoss(features=features, temperature=temperature)
                loss_+=total_loss.numpy()
            
            gradients = tape.gradient(total_loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        
        print(f"{epoch}). Loss : {loss_}")

        #to obtain test value if train test split
        if X_test is not None : 
            for pre_index_test in range (len(X_test)//batch_size+1):
                c_idx_test=np.arange(pre_index_test*batch_size, min(len(X_test), (pre_index_test+1)*batch_size))
                if len(c_idx_test)==0:
                    continue
                c_idx_test=idx_test[c_idx_test]
                c_inp_test=X_test[c_idx_test]
                if noise is None or noise == 0 :
                    input1_test=tf.convert_to_tensor(c_inp_test, dtype=tf.float32)
                    input2_test=tf.convert_to_tensor(c_inp_test, dtype=tf.float32)
                else : 
                    noise_vec_test = np.random.normal(loc=0, scale=noise, size=c_inp_test.shape)
                    input1_test = tf.convert_to_tensor(c_inp_test + noise_vec_test, dtype=tf.float32)
                    noise_vec_test = np.random.normal(loc=0, scale=noise, size=c_inp_test.shape)
                    input2_test = tf.convert_to_tensor(c_inp_test + noise_vec_test, dtype=tf.float32)
            
                with tf.GradientTape() as tape :
                    anchors_output_test=model(input1_test,  training=True)
                    neighbors_output_test=model(input2_test,  training=True)
                    features_test = tf.stack([anchors_output_test, neighbors_output_test], axis=1)
                    total_val_loss= SupConLoss(features=features_test, temperature=temperature)
                    val_loss_+=total_val_loss.numpy()
        print(f"val_loss : {val_loss_}")
        
        losses.append(loss_)
        val_losses.append(val_loss_)
        

        if early_stopping and X_test is not None:
            indice_max=val_losses.index(min(val_losses))
            if abs(indice_max-len(val_losses)-1)>5:
                break
            
    if plot_loss:
        x=[i for i in range (len(losses))]
        plt.plot(x, losses)
        plt.plot(x, val_losses)
        plt.legend(labels=['loss', 'val loss'])
        plt.plot()
    
    if evaluate_training and Y is not None :
        #result = model.predict(X)
        res, result, n_cluster, cluster_centers =cluster_embedding(model, X, Y, method=method, res=res, n=n, plot=False)
        ca=np.round(cluster_acc(Y, result), 5)
        nmi=np.round(normalized_mutual_info_score(Y, result), 5)
        ari=np.round(adjusted_rand_score(Y, result), 5)
        print( f"ARI {ari}, NMI: {nmi}, CA : {ca}, clusters : {n_cluster}")
        history["res"]=res
        history["clusters"]=n_cluster
        history['NMI final']=nmi
        history['ARI final']=ari
        history['CA final']=ca
    
    if evaluate_training and Y_test is not None :
        test_embedding=model.predict(X_test, verbose=0)
        distances=euclidean_distances(test_embedding, cluster_centers)
        test_clusters=np.argmin(distances, axis=1)
        val_ca=np.round(cluster_acc(Y_test, test_clusters), 5)
        val_nmi=np.round(normalized_mutual_info_score(Y_test, test_clusters), 5)
        val_ari=np.round(adjusted_rand_score(Y_test, test_clusters), 5)
        print( f"val_ARI {val_ari}, val_NMI: {val_nmi}, val_CA : {val_ca}")
        history['val_NMI final']=val_nmi
        history['val_ARI final']=val_ari
        history['val_CA final']=val_ca
                
    history["loss"]=losses
    history["val_loss"]=val_losses
    
    history['final_loss']=loss_
    history['final_val_loss']=val_loss_
        
    #save dictionnary
    save_results_to_csv(history, "data/contrastive-sc/random_search/contrastive_sc_random_search_2.csv")

    return model, result

In [ ]:
for i in range (100):
    lr, dropout, batch_size, noise, temperature=sample_hyperparameters(lr_space=[0.004,0.04,0.4,4.0, 0.1,0.2,0.3,0.5,0.6,0.7,0.8],
                           dropout_space=[i/10 for i in range (7,10)],
                           batch_size_space=[32,64,128,256,512,1024], 
                            noise_space=[0,0.01,0.05,0.1,0.5,1.0],
                            temperature_space=[0.01,0.02,0.05,0.07,0.1,0.2,0.5,0.7,1])
    model, results=train_model(x_train, Y=y_train, n='auto', res='auto', nb_epochs=60, lr=lr, 
                temperature=temperature, dropout=dropout, evaluate_training=True,  eval_interval=1, 
                layers=[256, 64, 32], noise=noise, method='leiden', batch_size=batch_size, 
               X_test=x_val, Y_test=y_val, early_stopping=True, plot_loss=False)

13). Loss : 17.120649367570877
val_loss : 5.6755316480994225
14). Loss : 17.14497748017311
val_loss : 5.974091640673578
15). Loss : 15.280142799019814
val_loss : 5.77702128875535
16). Loss : 14.851214721798897
val_loss : 5.646068215370178
17). Loss : 14.70672331750393
val_loss : 5.786845583468676
18). Loss : 13.786629781126976
val_loss : 5.140639309372318
19). Loss : 13.082439005374908
val_loss : 5.121720978291705
20). Loss : 13.182932063937187
val_loss : 5.553546100854874
21). Loss : 12.691943660378456
val_loss : 5.567655688151717
22). Loss : 12.427534207701683
val_loss : 5.601148154004477
23). Loss : 12.433687254786491
val_loss : 5.7195089012384415
La résolution est de :  0.09
ARI 0.72088, NMI: 0.74959, CA : 0.7139, clusters : 4
val_ARI 0.631, val_NMI: 0.60804, val_CA : 0.66693


Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 2000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 2000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 256)            │       512,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_30 (ReLU)                 │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_31 (ReLU)                 │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_16 (Lambda)              │ (None, 32)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 532,064 (2.03 MB)

 Trainable params: 531,424 (2.03 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_36 (InputLayer)     │ (None, 2000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 2000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ (None, 256)            │       512,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_72          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_72 (ReLU)                 │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_109 (Dense)               │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_73          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_73 (ReLU)                 │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_110 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_37 (Lambda)              │ (None, 32)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 532,064 (2.03 MB)

 Trainable params: 531,424 (2.03 MB)

 Non-trainable params: 640 (2.50 KB)

0). Loss : 82.54757285118103
val_loss : 14.827123492956161
1). Loss : 63.99666202068329
val_loss : 12.736982211470604
2). Loss : 51.95105159282684
val_loss : 10.701116502285004
3). Loss : 41.620741844177246
val_loss : 8.47328908102645
4). Loss : 35.148372173309326
val_loss : 7.52818099106662
5). Loss : 30.272596776485443
val_loss : 7.063734412193298
6). Loss : 27.022318840026855
val_loss : 6.462758779525757
7). Loss : 24.233322858810425
val_loss : 6.1528272799082515
8). Loss : 23.190501809120178
val_loss : 5.683576243260177
9). Loss : 20.61766368150711
val_loss : 5.610681363497861
10). Loss : 19.583632946014404
val_loss : 5.532586727873422
11). Loss : 18.171755760908127
val_loss : 5.7399157881736755
12). Loss : 17.99454239010811
val_loss : 4.950809470261447
13). Loss : 16.131360977888107
val_loss : 4.550481566366216
14). Loss : 15.251209676265717
val_loss : 4.693189655430615
15). Loss : 14.767751902341843
val_loss : 4.654785147735083
16). Loss : 14.200105607509613
val_loss : 4.42031237